In [2]:
# !pip install pytorch-transformers

In [2]:
#imports all the libraries we need
import torch
import os
import string
import copy
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from pytorch_transformers import *
import numpy as np
import json
import collections
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [3]:
# main the tool to change sentences into vector representations
tokenizer = RobertaTokenizer.from_pretrained('roberta-base') 

In [4]:
#load_dabasically reads in data --> takes in everything from json files
def load_data(filename):
  data = []
  # read in each line and add it to list
  with open(filename, mode = "r") as file:
      for line in file:
          data.append(json.loads(line))
  return data

#Takes a list of words (strings) and a sentence (as a RoBERTa tokenized ID list) and returns a list
#of pairs indicating the tokens' start and end positions in the sentence for each word
      
def find_word_in_tokenized_sentence(word,token_ids): #gets the same word in a sentence
  decomposedWord = tokenizer.encode(word)
  # Iterate through to find a matching sublist of the token_ids
  for i in range(len(token_ids)):
    if token_ids[i] == decomposedWord[0] and token_ids[i:i+len(decomposedWord)] == decomposedWord:
      return (i,i+len(decomposedWord)-1) #matching word found
  # if no same word --> return this 
  return (-1,-1)
  
def find_words_in_tokenized_sentences(wordList,token_ids): #gets list of word positions in a sentence
  intList = []
  for word in wordList:
    if len(intList) == 0:
      intList.append(find_word_in_tokenized_sentence(word,token_ids))
    else:
      afterLastInterval = intList[-1][1]+1
      interv = find_word_in_tokenized_sentence(word,token_ids[afterLastInterval:])
      actualPositions = (interv[0] + afterLastInterval,interv[1]+afterLastInterval)
      intList.append(actualPositions)
  return intList #returns list of positions

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels, return_predict_correctness = False):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  if return_predict_correctness:
    return np.sum(pred_flat == labels_flat) / len(labels_flat), pred_flat == labels_flat
  else:
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_predictions(preds):
  pred_flat = np.argmax(preds, axis=1).flatten()
  return pred_flat == 1

In [5]:
# check = torch.cuda.is_available() --> never could get cuda to work so nvm on that
# print(check)

BATCH_SIZE = 32 #decreased the size until the CPU stops dying
EPOCHS = 10 #could do more for higher accuracy buts takes too long
PATIENCE = 10

In [6]:
# Create a function to preprocess the WiC data
def wic_preprocessing(json_objects, training = True, shuffle_data = False, verbose = False):
  wic_sentences = [] #create arrays for sentences, encoded, etc. 
  wic_encoded = []
  wic_labels = []
  wic_word_locs = []
  wic_indexes = []
  for index, example in enumerate(json_objects):
    #wic_indexes.append(example['idx']) # Is it the index??
    wic_indexes.append(index)
    sentence = f"<s>{example['sentence1']}</s><s>{example['sentence2']}</s>"
    wic_sentences.append(sentence)
    # Then encode the sentences
    wic_encoded.append(tokenizer.encode(sentence, add_special_tokens=False))
    # Find the word in each sentences
    word = example['word']
    word_locs = (-1, -1)
    # Split the 2 sentences on space. (Also, lemmatize and uncapitilize each word)
    sent1_split = example['sentence1'].split(' ')
    sent2_split = example['sentence2'].split(' ')
    # Get the index of word in both sentences
    sent1_word_char_loc = (example['start1'], example['end1'])
    sent2_word_char_loc = (example['start2'], example['end2'])
    # Create a variable to keep track of the number of characters parsed in each sentence as we loop
    sent_chars = 0
    # Loop over the words in the first sentence
    i, j = 0, 0
    word1_not_found, word2_not_found = True, True
    while word1_not_found and i < len(sent1_split):
      word_len = len(sent1_split[i])
      if sent_chars >= sent1_word_char_loc[0] or sent_chars + word_len >= sent1_word_char_loc[1]:
        word_locs = (i, -1) # Found the word in the sentence
        word1_not_found = False
      elif sent_chars > sent1_word_char_loc[1]:
        # If we somehow got past the word. Assume it was the previous word
        word_locs = (i - 1, -1) # Found the word in the sentence
        word1_not_found = False
      else:
        # Look at the next word
        sent_chars += word_len + 1 # Plus one for the space
        i += 1
    # Loop over the words in the second
    sent_chars = 0 # Reset
    while word2_not_found and j < len(sent2_split):
      word_len = len(sent2_split[j])
      if sent_chars >= sent2_word_char_loc[0] or sent_chars + word_len >= sent2_word_char_loc[1]:
        word_locs = (i, j) # Found the word in the sentence
        word2_not_found = False
      elif sent_chars > sent2_word_char_loc[1]:
        # If we somehow got past the word. Assume it was the previous word
        word_locs = (i, j - 1) # Found the word in the sentence
        word2_not_found = False
      else:
        # Look at the next word
        sent_chars += word_len + 1 # Plus one for the space
        j += 1
    # For testing
    if verbose:
      print(word)
      print(sent1_split)
      print(sent2_split)
      print(word_locs)
    # Now to find the word in the tokenized sentences
    word1 = sent1_split[word_locs[0]].translate(str.maketrans('', '', string.punctuation)) #Remove punctuation (See https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string)
    word2 = sent2_split[word_locs[1]].translate(str.maketrans('', '', string.punctuation)) #Remove punctuation
    token_word_locs = find_words_in_tokenized_sentences([word1, word2], wic_encoded[-1])
    wic_word_locs.append(token_word_locs)
    # Get the label if we expect it to be there
    if training:
      if example['label']:
        wic_labels.append(1)
      else:
        wic_labels.append(0)
  # Pad the sequences and find the encoded word location in the combined input
  max_len = np.array([len(ex) for ex in wic_encoded]).max()
  wic_padded = {"input_ids" : [], "attention_mask" : [], "token_type_ids" : [], "word1_locs": [], "word2_locs" : [], "index" : wic_indexes}
  for i in range(0, len(wic_encoded)):
    enc_sentence = wic_encoded[i]
    word_locs = wic_word_locs[i]
    # Pad the sequences
    ex_len = len(enc_sentence)
    padded_sentence = enc_sentence.copy()
    padded_sentence.extend([0]*(max_len - ex_len))
    wic_padded["input_ids"].append(padded_sentence)
    padded_mask = [1] * ex_len
    padded_mask.extend([0]*(max_len - ex_len))
    wic_padded["attention_mask"].append(padded_mask)
    # Create the vector to get back the words after RoBERTa
    token_word_locs = wic_word_locs[i]
    first_word_loc = []
    second_word_loc = []
    len_first_word = token_word_locs[0][1] - token_word_locs[0][0] + 1
    len_second_word = token_word_locs[1][1] - token_word_locs[1][0] + 1
    for j in range(0, max_len):
      if j >= token_word_locs[0][0] and j <= token_word_locs[0][1]:
        #Part of the first word
        first_word_loc.append(1.0 / len_first_word)
      else:
        first_word_loc.append(0.0)
      if j >= token_word_locs[1][0] and j <= token_word_locs[1][1]:
        #Part of the second word
        second_word_loc.append(1.0 / len_second_word)
      else:
        second_word_loc.append(0.0)
    # We want to append a [1, max_len] vector instead of a [max_len] vector so wrap in an array
    wic_padded["word1_locs"].append([first_word_loc])
    wic_padded["word2_locs"].append([second_word_loc])
    # token_type_ids is a mask that tells where the first and second sentences are
    token_type_id = []
    first_sentence = True
    sentence_start = True
    for token in padded_sentence:
      if first_sentence and sentence_start and token == 0:
        # Allows 0 at the start of the first sentence
        token_type_id.append(0)
      elif first_sentence and token > 0:
        if sentence_start:
          sentence_start = False
        token_type_id.append(0)
      elif first_sentence and not sentence_start and token == 0:
        first_sentence = False
        # Start of second sentence
        token_type_id.append(1)
      else:
        # Second sentence
        token_type_id.append(1)
    wic_padded["token_type_ids"].append(token_type_id)
  if training:
    if shuffle_data:
      # Shuffle the data
      raw_set = {"input_ids": [], "token_type_ids": [], "attention_mask": [], "labels": [], "word1_locs": [], "word2_locs" : [], "index" : []}
      raw_set["input_ids"], raw_set["token_type_ids"], raw_set["attention_mask"], raw_set["labels"], raw_set["word1_locs"], raw_set["word2_locs"], raw_set["index"] = shuffle(
          wic_padded["input_ids"], wic_padded["token_type_ids"], wic_padded["attention_mask"], wic_labels, wic_padded["word1_locs"], wic_padded["word2_locs"], wic_padded["index"])
    else:
      raw_set = {"input_ids": wic_padded["input_ids"], "token_type_ids": wic_padded["token_type_ids"],
                 "attention_mask": wic_padded["attention_mask"], "labels": wic_labels, "index" : wic_padded["index"],
                 "word1_locs": wic_padded["word1_locs"], "word2_locs" : wic_padded["word2_locs"]}
  else: # No labels present (Testing set)
    # Do not shuffle the testing set
    raw_set = {"input_ids": wic_padded["input_ids"], "token_type_ids": wic_padded["token_type_ids"], 
               "attention_mask": wic_padded["attention_mask"], "index" : wic_padded["index"], 
               "word1_locs": wic_padded["word1_locs"], "word2_locs" : wic_padded["word2_locs"]}
  # Return the raw data (Need to put them in a PyTorch tensor and dataset)
  return raw_set

In [7]:
# Process the data
train_json_objs = load_data("train.jsonl")
raw_train_set = wic_preprocessing(train_json_objs, shuffle_data=True, verbose = False) # We do not want to shuffle for now.
print(train_json_objs[raw_train_set["index"][15]])
print(raw_train_set["input_ids"][15]),
print(raw_train_set["token_type_ids"][15]),
print(raw_train_set["attention_mask"][15]),
print(raw_train_set["labels"][15])
print(raw_train_set["word1_locs"][15]) #right, so it's better to use one-hot vectors to do a selection afterwards, I can compute it more precisely with the token intervals however
print(raw_train_set["word2_locs"][15])

{'word': 'clash', 'sentence1': 'These colors clash.', 'sentence2': 'Fans from opposing teams clashed on the streets after the game.', 'idx': 3356, 'label': True, 'start1': 13, 'start2': 25, 'end1': 18, 'end2': 32, 'version': 1.1}
[0, 1216, 8089, 6064, 4, 2, 0, 8846, 31, 9375, 893, 18789, 15, 5, 2827, 71, 5, 177, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1
[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [8]:
print(len(raw_train_set["labels"])/BATCH_SIZE)

169.625


In [9]:
# Create a PyTorch dataset for it
train_data = TensorDataset(
    torch.tensor(raw_train_set["input_ids"]),
    torch.tensor(raw_train_set["token_type_ids"]),
    torch.tensor(raw_train_set["attention_mask"]),
    torch.tensor(raw_train_set["labels"]),
    torch.tensor(raw_train_set["word1_locs"]),
    torch.tensor(raw_train_set["word2_locs"]),
    torch.tensor(raw_train_set["index"])
)
# Create a sampler and loader
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [10]:
# Load the json objects from each file
test_json_objs = load_data("test.jsonl")
valid_json_objs = load_data("val.jsonl")
# Process the objects
raw_test_set = wic_preprocessing(test_json_objs, training = False) # The labels for the testing set are unknown
raw_valid_set = wic_preprocessing(valid_json_objs)
# Create PyTorch datasets
test_data = TensorDataset(
    torch.tensor(raw_test_set["input_ids"]),
    torch.tensor(raw_test_set["token_type_ids"]),
    torch.tensor(raw_test_set["attention_mask"]),
    torch.tensor(raw_test_set["word1_locs"]),
    torch.tensor(raw_test_set["word2_locs"]),
    torch.tensor(raw_test_set["index"])
)
validation_data = TensorDataset(
    torch.tensor(raw_valid_set["input_ids"]),
    torch.tensor(raw_valid_set["token_type_ids"]),
    torch.tensor(raw_valid_set["attention_mask"]),
    torch.tensor(raw_valid_set["labels"]),
    torch.tensor(raw_valid_set["word1_locs"]),
    torch.tensor(raw_valid_set["word2_locs"]),
    torch.tensor(raw_valid_set["index"])
)
# Create a sampler and loader for each
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE);

In [11]:
# Variable for the path to the model
NAME = 'NewModel'
PATH = f'NLP-WiC/WiCBaseline'
# if not os.path.exists(PATH):
#  os.mkdir(PATH) # Make the directory

# Load the base RoBERTa model
#model = RobertaModel.from_pretrained('roberta-base')


model = RobertaForMaskedLM.from_pretrained('roberta-base')
roberta_init_weights = model.state_dict()


# This is where the model weights would be loaded for our fine-tuned models
#Model_name = "RoBERTa_0.0_1.0_0.0_Epochs2"
#PATH = f'/content/drive/My Drive/CSI5138_Project/Models/{NAME}/{Model_name}'
#model = RobertaModel.from_pretrained(PATH)
#model = RobertaForMaskedLM.from_pretrained(PATH)

In [12]:
class WiC_Head(torch.nn.Module):
    def __init__(self, roberta_based_model, embedding_size = 768):
        """
        Keeps a reference to the provided RoBERTa model. 
        It then adds a linear layer that takes the distance between two 
        """
        super(WiC_Head, self).__init__()
        self.embedding_size = embedding_size
        self.embedder = roberta_based_model
        self.linear_diff = torch.nn.Linear(embedding_size, 250, bias = True)
        self.linear_seperator = torch.nn.Linear(250, 2, bias = True)
        self.loss = torch.nn.CrossEntropyLoss()
        self.activation = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

    def forward(self, input_ids=None, attention_mask=None, labels=None,
                word1_locs = None, word2_locs = None):
        """
        Takes in the same argument as RoBERTa forward plus two tensors for the location of the 2 words to compare
        """
        if word1_locs is None or word2_locs is None:
          raise ValueError("The tensors (word1_locs, word1_locs) containing the location of the words to compare in the input vector must be provided.")
        elif input_ids is None:
          raise ValueError("The input_ids tensor must be provided.")
        elif word1_locs.shape[0] != input_ids.shape[0] or word2_locs.shape[0] != input_ids.shape[0]:
          raise ValueError("All provided vectors should have the same batch size.")
        batch_size = word1_locs.shape[0]
        # Get the embeddings
        embs, _ = self.embedder.roberta(input_ids=input_ids, attention_mask=attention_mask)
        # Get the words
        word1s = torch.matmul(word1_locs, embs).view(batch_size, self.embedding_size)
        word2s = torch.matmul(word2_locs, embs).view(batch_size, self.embedding_size)
        diff = word1s - word2s
        # Calculate outputs using activation
        layer1_results = self.activation(self.linear_diff(diff))
        logits = self.softmax(self.linear_seperator(layer1_results))
        outputs = logits
        # Calculate the loss
        if labels is not None:
            #  We want seperation like a SVM so use Hinge loss
            loss = self.loss(logits.view(-1, 2), labels.view(-1))
            outputs = (loss, logits)
        return outputs

In [13]:
class_model = WiC_Head(model, embedding_size = 768)

In [ ]:
# Variable for minimal accuracy
MIN_ACCURACY = 0.65 # Based on the average accuracy
REACHED_MIN_ACCURACY = False
best_weights = class_model.state_dict()
# Want to maximize accuracy
max_val_acc = (0, 0)
# Put the model in GPU
# class_model.cuda()
# Create the optimizer
param_optimizer = list(class_model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
# I use the one that comes with the models, but any other optimizer could be used
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
# Store our loss and accuracy for plotting
fit_history = {"loss": [],  "accuracy": [], "val_loss": [], "val_accuracy": []}
epoch_number = 0
epoch_since_max = 0
continue_learning = True
while epoch_number < EPOCHS and continue_learning:
  epoch_number += 1
  print(f"Training epoch #{epoch_number}")
  # Tracking variables
  tr_loss, tr_accuracy = 0, 0
  nb_tr_examples, nb_tr_steps = 0, 0
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  # Training
  # Set our model to training mode (as opposed to evaluation mode)
  class_model.train()
  # Freeze RoBERTa weights
  #class_model.embedder.eval()
  class_model.embedder.requires_grad_ = False
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    # batch = tuple(t.cuda() for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_token_ids, b_input_mask, b_labels, b_word1, b_word2, b_index = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    #loss, logits = class_model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)   
    loss, logits = class_model(b_input_ids, attention_mask=b_input_mask, 
                               labels=b_labels, word1_locs = b_word1, word2_locs = b_word2) 
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()
    # Calculate the accuracy
    b_accuracy = flat_accuracy(logits, label_ids) # For RobertaForClassification
    # Append to fit history
    fit_history["loss"].append(loss.item()) 
    fit_history["accuracy"].append(b_accuracy) 
    # Update tracking variables
    tr_loss += loss.item()
    tr_accuracy += b_accuracy
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
    if nb_tr_steps%10 == 0:
      print("\t\tTraining Batch {}: Loss: {}; Accuracy: {}".format(nb_tr_steps, loss.item(), b_accuracy))
  print("Training:\n\tLoss: {}; Accuracy: {}".format(tr_loss/nb_tr_steps, tr_accuracy/nb_tr_steps))
  # Validation
  # Put model in evaluation mode to evaluate loss on the validation set
  class_model.eval()
  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    # batch = tuple(t.cuda() for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_token_ids, b_input_mask, b_labels, b_word1, b_word2, b_index = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      #loss, logits = class_model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)
      loss, logits = class_model(b_input_ids, attention_mask=b_input_mask, 
                                 labels=b_labels, word1_locs = b_word1, word2_locs = b_word2)
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.cpu().numpy()
    # Calculate the accuracy
    b_accuracy = flat_accuracy(logits, label_ids) # For RobertaForClassification
    # Append to fit history
    fit_history["val_loss"].append(loss.item()) 
    fit_history["val_accuracy"].append(b_accuracy) 
    # Update tracking variables
    eval_loss += loss.item()
    eval_accuracy += b_accuracy
    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1
    if nb_eval_steps%10 == 0:
      print("\t\tValidation Batch {}: Loss: {}; Accuracy: {}".format(nb_eval_steps, loss.item(), b_accuracy))
  eval_acc = eval_accuracy/nb_eval_steps
  if eval_acc >= max_val_acc[0]:
    max_val_acc = (eval_acc, epoch_number)
    continue_learning = True
    epoch_since_max = 0 # New max
    best_weights = copy.deepcopy(class_model.state_dict()) # Keep the best weights
    # See if we have reached min_accuracy
    if eval_acc >= MIN_ACCURACY:
      REACHED_MIN_ACCURACY = True
    # Save to file only if it has reached min acc
    if REACHED_MIN_ACCURACY:
      # Save the best weights to file
      torch.save(best_weights, os.path.join(PATH,'WiCHead.pt'))
      continue_learning = False # Stop learning. Reached baseline acc for this model
  else:
    epoch_since_max += 1
    if epoch_since_max > PATIENCE:
      continue_learning = False # Stop learning, starting to overfit
  print("Validation:\n\tLoss={}; Accuracy: {}".format(eval_loss/nb_eval_steps, eval_accuracy/nb_eval_steps))
print(f"Best accuracy ({max_val_acc[0]}) obtained at epoch #{max_val_acc[1]}.")
# Reload the best weights (from memory)
class_model.load_state_dict(best_weights)

Training epoch #1


/tmp/ipykernel_5273/1635428726.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = self.softmax(self.linear_seperator(layer1_results))
/home/nick/.local/lib/python3.8/site-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


In [ ]:
with open(os.path.join(PATH, "fit_history.json"), 'w') as json_file:
  json.dump(fit_history, json_file)

In [ ]:
# Save the best weights to file
torch.save(best_weights, os.path.join(PATH,'WiCHead.pt'))

In [ ]:
# Load the model
class_model.load_state_dict(torch.load(os.path.join(PATH,'WiCHead.pt')))
# Put the model in GPU
class_model.cuda()

In [ ]:
validation_predictions_correctness = {}
# Validation
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
# Put model in evaluation mode
class_model.eval()
# Evaluate data for one epoch
for batch in validation_dataloader:
  # Add batch to GPU
  batch = tuple(t.cuda() for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_token_ids, b_input_mask, b_labels, b_word1, b_word2, b_index = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    #loss, logits = class_model(b_input_ids, token_type_ids=b_token_ids, attention_mask=b_input_mask, labels=b_labels)
    loss, logits = class_model(b_input_ids, attention_mask=b_input_mask, 
                                labels=b_labels, word1_locs = b_word1, word2_locs = b_word2)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.cpu().numpy()
  # Calculate the accuracy
  b_accuracy, b_pred_correctness = flat_accuracy(logits, label_ids, return_predict_correctness = True) # For RobertaForClassification
  indexes = b_index.detach().cpu().numpy() # Get the indexes
  # Add to predictions
  for index, pred in zip(indexes, b_pred_correctness):
    validation_predictions_correctness[index] = pred
  # Update tracking variables
  eval_loss += loss.item()
  eval_accuracy += b_accuracy
  nb_eval_examples += b_input_ids.size(0)
  nb_eval_steps += 1
  if nb_eval_steps%10 == 0:
    print("\t\tValidation Batch {}: Loss: {}; Accuracy: {}".format(nb_eval_steps, loss.item(), b_accuracy))
print("Validation:\n\tLoss={}; Accuracy: {}".format(eval_loss/nb_eval_steps, eval_accuracy/nb_eval_steps))
validation_predictions_correctness = collections.OrderedDict(sorted(validation_predictions_correctness.items()))
print(validation_predictions_correctness)

In [ ]:
test_predictions = {}
nb_test_examples, nb_test_steps = 0, 0
# Testing
# Put model in evaluation mode to evaluate loss on the validation set
class_model.eval()
# Evaluate data for one epoch
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.cuda() for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_token_ids, b_input_mask, b_word1, b_word2, b_index = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = class_model(b_input_ids, attention_mask=b_input_mask, word1_locs = b_word1, word2_locs = b_word2)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  # Get the predictions
  b_preds = flat_predictions(logits)
  indexes = b_index.detach().cpu().numpy() # Get the indexes
  for index, pred in zip(indexes, b_preds):
    test_predictions[index] = pred
  # Update tracking variables
  nb_test_examples += b_input_ids.size(0)
  nb_test_steps += 1
  if nb_test_steps%10 == 0:
    print("\t\tTest Batch {}".format(nb_test_steps))
# Print final results
print("Testing Done!")
test_predictions = collections.OrderedDict(sorted(test_predictions.items()))
print(test_predictions)